In [1]:
import numpy
from collections import deque
import pandas
import math

In [2]:
def iterate_fund(ladder, yield_curve, max_maturity):
    reduce_maturity(ladder)
    
    payments = get_payments(ladder)

    sold_bond = ladder.popleft()
    payments += sold_bond.value(yield_curve)

    new_bond = Bond(payments, yield_curve[max_maturity-1], max_maturity)
    ladder.append(new_bond)
    
    # This happens *after* we sell the shortest bond and buy a new long one
    # (at least, that's what longinvest does...)
    nav = get_nav(ladder, yield_curve)

    return (ladder, payments, nav)

def get_nav(ladder, rates):
    return sum((b.value(rates) for b in ladder))

def get_payments(ladder):
    return sum((b.gen_payment() for b in ladder))

def reduce_maturity(ladder):
    for b in ladder:
        b.maturity -= 1
    return ladder

In [3]:
class Bond:
    def __init__(self, face_value, yield_pct, maturity):
        self.face_value = face_value
        self.yield_pct = yield_pct
        self.maturity = maturity
        
    def __repr__(self):
        return ('Maturity: %d | Yield: %.2f%% | Face Value: $%.2f' % (self.maturity, self.yield_pct * 100, self.face_value))
        
    def gen_payment(self):
        return self.face_value * self.yield_pct
    
    def value(self, rates):
        value = numpy.pv(rates[self.maturity - 1], self.maturity, self.gen_payment(), self.face_value)
        return -value

In [4]:
def bootstrap(yield_curve, max_bonds, min_maturity):
    bond_yield = yield_curve[max_bonds - 1]
    ladder = deque()
    starting_face_value = 50 # chosen arbitrarily (to match longinvest)

    for i, j in zip(range(max_bonds), range(min_maturity, max_bonds+1)):
        face_value = pow(1 + bond_yield, i) * starting_face_value
        b = Bond(face_value, bond_yield, j)
        ladder.append(b)
    return ladder
bootstrap([.0532]*10, 10, 2)

deque([Maturity: 2 | Yield: 5.32% | Face Value: $50.00,
       Maturity: 3 | Yield: 5.32% | Face Value: $52.66,
       Maturity: 4 | Yield: 5.32% | Face Value: $55.46,
       Maturity: 5 | Yield: 5.32% | Face Value: $58.41,
       Maturity: 6 | Yield: 5.32% | Face Value: $61.52,
       Maturity: 7 | Yield: 5.32% | Face Value: $64.79,
       Maturity: 8 | Yield: 5.32% | Face Value: $68.24,
       Maturity: 9 | Yield: 5.32% | Face Value: $71.87,
       Maturity: 10 | Yield: 5.32% | Face Value: $75.69])

In [5]:
HISTORICAL_RATES = pandas.read_csv('oecd_interest_rates.csv', index_col=0)
HISTORICAL_RATES.head()

,AUS,AUT,BEL,CAN,DNK,FRA,DEU,ITA,JPN,NLD,...,SWE,CHE,GBR,USA,ALL AVERAGE,STD-DEV,16 COUNTRIES AVERAGE,STD-DEV.1,NO JPN CHE AVERAGE,STD-DEV.2
YEAR,,,,,,,,,,,,,,,,,,,,,
1970,0.0665,0.0781,0.0771,0.0791,0.1057,0.0812,0.0830,0.0901,0.0719,0.0822,...,0.0739,0.0582,0.0922,0.0735,0.0784,0.0119,0.0784,0.0119,0.0783,0.0085
1971,0.0671,0.0771,0.0729,0.0695,0.1067,0.0779,0.0798,0.0834,0.0728,0.0735,...,0.0723,0.0527,0.0890,0.0616,0.0747,0.0126,0.0747,0.0126,0.0740,0.0080
1972,0.0583,0.0736,0.0696,0.0723,0.1037,0.0742,0.0786,0.0747,0.0669,0.0688,...,0.0729,0.0497,0.0890,0.0621,0.0718,0.0127,0.0718,0.0127,0.0714,0.0082
1973,0.0693,0.0825,0.0735,0.0756,0.1107,0.0833,0.0931,0.0742,0.0726,0.0792,...,0.0738,0.0560,0.1071,0.0684,0.0788,0.0151,0.0788,0.0151,0.0785,0.0121
1974,0.0904,0.0974,0.0857,0.0890,0.1455,0.1056,0.1037,0.0987,0.0926,0.0982,...,0.0778,0.0715,0.1477,0.0756,0.0967,0.0231,0.0967,0.0231,0.0951,0.0200


In [6]:
def build_yield_curve(raw_rate, yield_curve_size=10):
    s = pandas.Series(raw_rate, index=numpy.arange(yield_curve_size))
    return s.tolist()

In [7]:
['%.2f' % (s*100) for s in build_yield_curve(HISTORICAL_RATES.iloc[0]['AUS'])]

['6.65',
 '6.65',
 '6.65',
 '6.65',
 '6.65',
 '6.65',
 '6.65',
 '6.65',
 '6.65',
 '6.65']

In [8]:
bootstrap(build_yield_curve(HISTORICAL_RATES.iloc[0]['AUS']), 10, 4)

deque([Maturity: 4 | Yield: 6.65% | Face Value: $50.00,
       Maturity: 5 | Yield: 6.65% | Face Value: $53.33,
       Maturity: 6 | Yield: 6.65% | Face Value: $56.87,
       Maturity: 7 | Yield: 6.65% | Face Value: $60.65,
       Maturity: 8 | Yield: 6.65% | Face Value: $64.69,
       Maturity: 9 | Yield: 6.65% | Face Value: $68.99,
       Maturity: 10 | Yield: 6.65% | Face Value: $73.58])

In [9]:
def loop(ladder, rates, max_maturity, start_year, end_year):
    df = pandas.DataFrame(columns=['NAV', 'Payments', 'Change'], index=numpy.arange(start_year, end_year + 1))

    for (year, current_rates) in rates.iteritems():
        (ladder, payments, nav) = iterate_fund(ladder, build_yield_curve(current_rates), max_maturity)
        df.loc[year] = {'NAV' : nav, 'Payments' : payments}

    calculate_returns(df)
    return df

def calculate_returns(df):
    # Longinvest calculates the return based on comparison's to
    # next year's NAV. So I'll do the same. Even though that seems
    # weird to me. Maybe it's because the rates are based on January?
    # Hmmm...that sounds plausible.
    max_row = df.shape[0]

    for i in range(max_row - 1):
        next_nav = df.iloc[i+1]['NAV']
        nav = df.iloc[i]['NAV']
        change = (next_nav - nav) / nav
        df.iloc[i]['Change'] = change
    return df

def simulate(max_maturity, min_maturity, rates):
    """ This is just something to save on typing...and make clearer what the bounds on the fund are """
    # find the first non-NaN number in rates
    rates = rates.dropna()
    ladder = bootstrap(build_yield_curve(rates.iloc[0]), max_maturity, min_maturity)
    start_year = rates.head(1).index[0]
    end_year = rates.tail(1).index[0]
    return loop(ladder, rates, max_maturity, start_year, end_year)

# The "naive" approach

Other bond return simulations don't hold a ladder, they just sell the bond after a single year. That is, buy a 10-year bond, sell it after 1-year and buy another 10-year bond. That's the same as holding a single maturity (above) but with a 10-year maturity.

In [10]:
simulate(10, 10, HISTORICAL_RATES['SGD']).head()

,NAV,Payments,Change
1999,52.25,52.25,0.054505
2000,55.0979,55.0979,0.105152
2001,60.8915,60.8915,0.0378792
2002,63.1981,63.1981,0.0829729
2003,68.4418,68.4418,0.00531988


# Saving to CSV
To do anything useful, you probably want to save the results to a CSV file. Here's a commented-out example of how to do that.

In [21]:
countries = [
    'AUS',
    'AUT',
    'BEL',
    'CAN',
    'DNK',
    'FRA',
    'DEU',
    'ITA',
    'JPN',
    'NLD',
    'NOR',
    'SGD', # 1999 onward
    'ESP', # 1979 onward
    'SWE',
    'CHE',
    'GBR',
    'USA',
    'ALL AVERAGE',
    '16 COUNTRIES AVERAGE',
    'NO JPN CHE AVERAGE',
]

pd = pandas.DataFrame(columns=countries)

for c in countries:
    returns = simulate(10, 10, HISTORICAL_RATES[c])
    pd[c] = returns['Change']

pd.head()
#pd.to_csv('oecd_bond_returns.csv')